# Pytorch classifier notebook

V1 : only 1 split. First implementation  
V2 : with all folds

In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42)

DATASET_INPUT_FILE = 'train.csv'

FEATURES_LIST_TOTRAIN = ['feature_'+str(i) for i in range(130)]

#pd.set_option('display.max_rows', 500)

In [40]:
# Deterministic Behavior
seed = 42
#os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
#random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False

In [41]:
BATCH_SIZE = 50000
#BATCH_SIZE = 80000
NUM_EPOCHS = 100

In [42]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]
            

In [43]:
# This function accounts for variable instance counts in each split by dividing utility_pi by number of instances (but this has been removed)
# It also does some copy of dataframe to prevent memory overwrite
def utility_function(df_test, df_test_predictions):
    df_test_copy = df_test.copy(deep=True)
    df_test_copy.loc[:, 'utility_pj'] = df_test_copy['weight'] * df_test_copy['resp'] * df_test_predictions
    #df_test_utility_pi = df_test_copy.groupby('date')['utility_pj'].sum() / df_test_copy.groupby('date')['utility_pj'].count()
    df_test_utility_pi = df_test_copy.groupby('date')['utility_pj'].sum()

    nb_unique_dates = df_test_utility_pi.shape[0]
    t = (df_test_utility_pi.sum() / np.sqrt(df_test_utility_pi.pow(2).sum())) * (np.sqrt(250 / np.abs(nb_unique_dates)))
    u = min(max(t, 0), 6) * df_test_utility_pi.sum()
    del df_test_copy
    
    return(u)

In [44]:
torch.device

torch.device

In [45]:
torch.cuda.is_available()

True

In [46]:
torch.cuda.current_device()

0

In [47]:
torch.cuda.get_device_name(0)

'GeForce RTX 3090'

In [48]:
# Load data
    
df = pd.read_csv(DATASET_INPUT_FILE)
df['resp_positive'] = ((df['resp'])>0)*1  # Target to predict

print('Data loaded')


Data loaded


In [49]:
cv = PurgedGroupTimeSeriesSplit(
    n_splits=5,
    #n_splits=5,
    #max_train_group_size=150,
    max_train_group_size=180,
    group_gap=20,
    max_test_group_size=60
)

In [50]:
#train_index, test_index = next(cv.split(df, (df['resp'] > 0)*1, df['date']))

In [51]:
#(df.loc[train_index, 'resp'] > 0).astype(np.byte)

In [62]:
f_mean = df.loc[:, FEATURES_LIST_TOTRAIN].mean(axis=0)

In [63]:
f_mean.shape

(130,)

In [64]:
df.fillna(f_mean, inplace=True)

In [65]:
#print('Sum of model parameters:')
#[print(p.sum()) for p in model.parameters()]

In [66]:
patience=4

for fold, (train_index, test_index) in enumerate(cv.split(df, (df['resp'] > 0)*1, df['date'])):
    print('********* Fold : {}'.format(fold))
    
    ts_train = torch.tensor(df.loc[train_index, FEATURES_LIST_TOTRAIN].to_numpy(), device='cuda')
    ts_test = torch.tensor(df.loc[test_index, FEATURES_LIST_TOTRAIN].to_numpy(), device='cuda')
    ts_train_y = torch.tensor((df.loc[train_index, 'resp'] > 0).astype(np.byte).to_numpy(), device='cuda')
    ts_test_y = torch.tensor((df.loc[test_index, 'resp'] > 0).astype(np.byte).to_numpy(), device='cuda')
  
    train_dataset = torch.utils.data.TensorDataset(ts_train, ts_train_y)
    test_dataset = torch.utils.data.TensorDataset(ts_test, ts_test_y)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True) # pin_memory : VOIR RESULTAT
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True)
 
    torch.manual_seed(seed)

    model = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(len(FEATURES_LIST_TOTRAIN), 130),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(130, 130),
        nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(130, 130),
        nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(130, 130),
        nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(130, 130),
        nn.BatchNorm1d(130),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(130, 50),
        nn.BatchNorm1d(50),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(50, 1),
        nn.Sigmoid(),
    ).double().to('cuda')
    
    print('Number of model parameters :')
    numel_list = [p.numel() for p in model.parameters()]
    sum(numel_list), numel_list

    loss_fn = nn.BCELoss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
  
    model.eval()
    start_accuracy = accuracy_score(ts_test_y.cpu().numpy(), (model(ts_test).squeeze() > 0.5).cpu().numpy())
    start_utility_score = utility_function(df.loc[test_index], (model(ts_test).squeeze() > 0.5).cpu().numpy())
    print('Start Validation Accuracy: {:.4f}'.format(start_accuracy))
    print('Start Validation Utility: {:.4f}'.format(start_utility_score))

    Val_Loss = 0
    N_Samples = 0

    the_last_loss = 100
    the_last_utility_score = 0
    the_last_accuracy = 0
    trigger_times=0
    early_stopping_met = False
    
    for epoch in range(NUM_EPOCHS): 
        running_loss = 0.0        
        model.train()

        for batch in train_loader:
            #inputs, labels = batch[0], batch[1]
            inputs, labels = batch[0].to(device), batch[1].to(device) # A essayer
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = loss_fn(outputs, labels.unsqueeze(-1).double())
                loss.backward()
                optimizer.step()

        # update local train loss
            running_loss += loss.item() * inputs.size(0)

        # update global train loss
        epoch_loss = running_loss / len(train_loader.dataset)
        print('Epoch({}) - Training Loss: {:.4f}'.format(epoch, epoch_loss))

        # Validation 
        model.eval()
        vrunning_loss = 0.0
        num_samples = 0

        for batch in test_loader:
            #inputs, labels = batch[0], batch[1]
            inputs, labels = batch[0].to(device), batch[1].to(device)

            optimizer.zero_grad()
            with torch.no_grad():
                outputs = model(inputs)
                loss = loss_fn(outputs, labels.unsqueeze(-1).double())

            vrunning_loss += loss.item() * inputs.size(0)
            num_samples += labels.size(0)

        # update epoch loss
        vepoch_loss = vrunning_loss/num_samples
        print('Epoch({}) - Validation Loss: {:.4f}'.format(epoch, vepoch_loss))

        #print(f'Sum of model parameters ({epoch}):')
        #[print(p.sum()) for p in model.parameters()]

        model.eval()
        with torch.no_grad():
            vepoch_accuracy = accuracy_score(ts_test_y.cpu().numpy(), (model(ts_test).squeeze() > 0.5).cpu().numpy())
        print('Epoch({}) - Validation Accuracy: {:.4f}'.format(epoch, vepoch_accuracy))

        model.eval()
        with torch.no_grad():
            vepoch_utility_score = utility_function(df.loc[test_index], (model(ts_test).squeeze() > 0.5).cpu().numpy())
        print('Epoch({}) - Validation Utility score: {:.4f}'.format(epoch, vepoch_utility_score))

        # Check if Early Stopping
        #if vepoch_loss > the_last_loss:
        if (vepoch_utility_score < the_last_utility_score) and (vepoch_loss > the_last_loss) and (vepoch_accuracy < the_last_accuracy):
            trigger_times += 1

            print(f'Intermediate early stopping : vepoch_loss = {vepoch_loss:.4f}, the_last_loss={the_last_loss:.4f}')
            print(f'Intermediate early stopping : vepoch_accuracy = {vepoch_accuracy:.4f}, the_last_utility_score={the_last_accuracy:.4f}')
            print(f'Intermediate early stopping : vepoch_utility_score = {vepoch_utility_score:.4f}, the_last_utility_score={the_last_utility_score:.4f}')

            if trigger_times >= patience:
                print('Meet Early stopping!')
                early_stopping_met = True
                ##torch.save(model.state_dict(), f'model_{fold}.pt')
                break
        else:
            trigger_times = 0
            the_last_loss = vepoch_loss
            the_last_utility_score = vepoch_utility_score
            the_last_accuracy = vepoch_accuracy

            # Save model for the best version so far
            print(f'Saving model corresponding to last_utility_score == {the_last_utility_score}')
            torch.save(model.state_dict(), f'model_NN_fold{fold}_V2.pt')

        print('\n')
        
    # Update global loss
    Val_Loss += vepoch_loss * num_samples

    # Update global # of samples 
    N_Samples += num_samples

    if (early_stopping_met == False):
        print("Didn't meet early stopping : saving final model")
        # Save model if don't meet early stopping
        torch.save(model.state_dict(), f'model_NN_fold{fold}_V2.pt')

********* Fold : 0
Number of model parameters :
Start Validation Accuracy: 0.5035
Start Validation Utility: -0.0000
Epoch(0) - Training Loss: 0.6990
Epoch(0) - Validation Loss: 0.6933
Epoch(0) - Validation Accuracy: 0.5035
Epoch(0) - Validation Utility score: -0.0000
Saving model corresponding to last_utility_score == -0.0


Epoch(1) - Training Loss: 0.6947
Epoch(1) - Validation Loss: 0.6926
Epoch(1) - Validation Accuracy: 0.5105
Epoch(1) - Validation Utility score: 6.7237
Saving model corresponding to last_utility_score == 6.723662620373936


Epoch(2) - Training Loss: 0.6936
Epoch(2) - Validation Loss: 0.6925
Epoch(2) - Validation Accuracy: 0.5129
Epoch(2) - Validation Utility score: 48.1673
Saving model corresponding to last_utility_score == 48.16727546938743


Epoch(3) - Training Loss: 0.6929
Epoch(3) - Validation Loss: 0.6923
Epoch(3) - Validation Accuracy: 0.5133
Epoch(3) - Validation Utility score: 76.8659
Saving model corresponding to last_utility_score == 76.86586892469407


Ep

KeyboardInterrupt: 

In [ ]:
model.eval()
accuracy_score(ts_test_y.cpu().numpy(), (model(ts_test).squeeze() > 0.5).cpu().numpy())

In [ ]:
model.eval()
utility_function(df.loc[test_index], (model(ts_test).squeeze() > 0.5).cpu().numpy())

In [ ]:
model_load = model = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(len(FEATURES_LIST_TOTRAIN), 130),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(130, 130),
    nn.BatchNorm1d(130),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(130, 130),
    nn.BatchNorm1d(130),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(130, 130),
    nn.BatchNorm1d(130),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(130, 130),
    nn.BatchNorm1d(130),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(130, 50),
    nn.BatchNorm1d(50),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(50, 1),
    nn.Sigmoid(),
).double().to('cuda')

model_load.load_state_dict(torch.load(f'model_NN_fold1_V2.pt',map_location=torch.device('cuda')))

model_load.eval()
print(accuracy_score(ts_test_y.cpu().numpy(), (model_load(ts_test).squeeze() > 0.5).cpu().numpy()))

model_load.eval()
print(utility_function(df.loc[test_index], (model_load(ts_test).squeeze() > 0.5).cpu().numpy()))